In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
cv_data_usa = pd.read_csv('cv-dev-usa.csv', parse_dates=["Date"], index_col="Date").dropna()
cv_data_world = pd.read_csv('cv-dev-world.csv', parse_dates=["Date"], index_col="Date").dropna()

cv_data = cv_data_usa


In [ ]:
dead = cv_data.Dead
recovered = cv_data.Recovered

df = (pd.DataFrame({"Dead":dead
                   , "Recovered" : recovered
                  }
                  , index=cv_data.index)
          .resample("1W")
          .max()
     )

df.plot(kind='line'
        , figsize=(14, 6)
        , legend=True
        , label="Completed Cases"
        , grid = True
        , color = ['red', 'green']
        , title = "Death/Recovery Update"
        )
# Weekly
df.resample("1W").max().plot(kind='bar'
        , stacked=True
        , figsize=(14, 6)
        , legend=True
        , label="Completed Cases"
        , grid = True
        , align = 'center'
        , color = ['red', 'green']
        , title = "Death/Recovery Weekly Update"
        )
plt.gcf().autofmt_xdate()
plt.show()



In [ ]:



# plt.figure(num=None, figsize=(14,6), dpi=80, facecolor='#dddddd', edgecolor='k')
def growth_pcnt(ser): 
    return (ser - ser.shift(1)) / ser * 100

df = pd.DataFrame(growth_pcnt(cv_data.Dead), index=cv_data.index)
df.plot(
    kind = 'line'
    , title = 'Added Deaths Rate - full range (%)'
    , figsize=(14, 6)
    , grid = True
    , style = '.-'
    , color = ['brown']
)

df[30:].plot(
    kind = 'line'
    , title = 'The Rate of Added Deaths (%)'
    , figsize=(14, 6)
    , grid = True
    , style = '.-'
#     , color = ['brown']
)
plt.show()



In [ ]:
# Death rate in completed cases
death_rate = pd.DataFrame(dead / (cv_data.Dead + cv_data.Recovered) * 100.0
                          , index=cv_data.index)
death_rate.plot(
    kind = 'line'
    , title = 'Deaths/Completed Rate (%)'
    , figsize=(14, 6)
    , grid = True
    , style = '.-'
    , color = ['brown']
)
print(death_rate.tail(3))
plt.show()

In [ ]:
# Deaths daily additions
df = (cv_data.Dead - cv_data.Dead.shift(1)).dropna()
df.shift(30).dropna().plot(
    title="Daily Death Addition"
    , figsize=(14, 6)
    , grid = True
    , color = ['brown']
)
df.rolling(30).mean().dropna().plot(grid=True, legend=True, label="30 days MA")
plt.show()


In [ ]:
# Remaining open cases
df = (cv_data.Confirmed - cv_data.Dead - cv_data.Recovered).dropna()
df.plot(
    title="Remaining Open Cases"
    , figsize=(14, 6)
    , grid = True
    , color = ['blue']
)


In [ ]:
# import numpy as np
# from matplotlib import pyplot as plt
# import pandas as pd
# cv_data = pd.read_csv('cv-dev-usa.csv', parse_dates=True, index_col=0)

confirmed_new = (cv_data.Confirmed - cv_data.Confirmed.shift(1)).dropna()
dead_new = (cv_data.Dead - cv_data.Dead.shift(1)).dropna()
recovered_new = (cv_data.Recovered - cv_data.Recovered.shift(1)).dropna()

growth = pd.Series((confirmed_new - (dead_new + recovered_new)).array
                   , index=cv_data.index[1:]).dropna()

growth.plot(figsize=(14, 6)
            , title="Added New Cases"
            , legend=True
            , label="New Cases"
            , style='.-')
# growth.rolling(10).mean().plot(grid=True, legend=True, label="10 days MA")
# growth.rolling(20).mean().plot(grid=True, legend=True, label="20 days MA")
growth.rolling(30).mean().plot(grid=True, legend=True, label="30 days MA")
print(growth.tail(5))
plt.show()


In [ ]:
# New Cases -> Deathes Correlation
# import numpy as np
# from matplotlib import pyplot as plt
# import pandas as pd

# Mortality to confirmed after offsetDays and for cumm_wnd_days
def calcCorrelation(cumm_wnd_days : int
                    , days_shift : int):
    confirmed = cv_data.Confirmed
    dead = cv_data.Dead
    # accumulate total confirmed during that "cumm_wnd_days" days
    cumm_conf = (confirmed - confirmed.shift(cumm_wnd_days)).dropna().shift(days_shift)

    # death accumulate for same windows but "offsetDays" later
    dead_shift = dead.shift(days_shift)
    cumm_dead = (dead_shift -  dead_shift.shift(cumm_wnd_days)).dropna()

    ratio = cumm_dead * 100 / cumm_conf
    return ratio.corr(cumm_conf) 


def mortalityToNewCasesRatio(cumm_wnd_days : int
                             , offset_days : int):
    confirmed = cv_data.Confirmed
    dead = cv_data.Dead
    # accumulate total confirmed during that "cummDaysWindow" days
    cumm_conf = confirmed - confirmed.shift(cumm_wnd_days)

    # death accumulate for same windows but "offset_days" later
    deadOffset = dead.shift(offset_days)
    cummDead = deadOffset -  deadOffset.shift(cumm_wnd_days)

    res = cummDead * 100 / cumm_conf
    return res

def correlation(cum_wnd_days: int):
    num_observ_days = len(cv_data.index) - 2 *cum_wnd_days
    after_days_arr = range(1, num_observ_days)
    
    corr_arr = [calcCorrelation(cum_wnd_days, after_days) 
               for after_days in after_days_arr]
    return pd.DataFrame(data={'Correlation': corr_arr}, index=after_days_arr)
    
def plot_correlation_chart(corr : pd.DataFrame
                          , cum_wnd_days: int):
    corr.plot(figsize=(14, 8)
             , title=f"Mortality/New Cases Day Shift correlation\nCumm Window = {cum_wnd_days} days"
             , legend=False, label="Correlation") 
    plt.ylabel("Correlation")
    plt.xlabel("Day Shift")
    plt.show()
    
def plot_cases_mortality_chart(cv_data, **kwargs):
    cumm_wnd_days = kwargs["window_size"]
    days_shift = kwargs["offset_days"]
    maDays = kwargs["ma_days"] if "ma_days" in kwargs else 5

    ratio = mortalityToNewCasesRatio(cumm_wnd_days
                                    , days_shift).dropna()
    ratio.plot(title=f"Offset: {days_shift} days\nCummulative Window: {cumm_wnd_days} days"
           , figsize=(14, 8)
           , grid=True
           , legend=True
           , label="Mortality to New Cases Ratio"
           )

    ratio.rolling(maDays).mean().plot(grid=True
                                  , label="{} days MA".format(maDays)
                                  , legend=True)
    plt.ylabel("Ratio (%)")
    plt.show()
    print(ratio.tail(5))

# --- main
# # for cum_wnd_days in [15, 20, 30]:
# #     plot_correlation_chart(confirmed, dead, cum_wnd_days)

cum_wnd_days = 15
corr = correlation(cum_wnd_days)
plot_cases_mortality_chart(cv_data
                           , window_size=cum_wnd_days
                           , offset_days=cum_wnd_days 
                           , ma_days=10)

plot_correlation_chart(corr, cum_wnd_days)
plot_cases_mortality_chart(cv_data
                           , window_size=cum_wnd_days
                           , offset_days=corr.idxmax()[0] # taking maximal correlation value
                           , ma_days=10)

